In [26]:
%matplotlib inline

This is testing the Nipype DTI-TK interfaces according to the following tutorial: http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Registration

Note: some text excerpts are copied directly from the DTI-TK documentation at the address, above, and have been adapted for the Nipype implementation

DTI-TK developed by Gary Hui Zhang, gary.zhang@ucl.ac.uk
You are running Release 2.3.3. For additional help, visit http://dti-tk.sf.net

In [27]:
import os
from glob import glob
import nipype.interfaces.dtitk as dtitk

In [28]:
#basepath = '/Users/kesshijordan/ref_data/interscan_dys/test_dtitk_2/sandbox/'
basepath = '.'
fixed_tensor = os.path.join(basepath, 'Ptensor.nii.gz')
moving_tensor = os.path.join(basepath, 'Ttensor.nii.gz')
fixed_vol = './Ptensor_tr.nii.gz'
moving_vol = './Ttensor_tr.nii.gz'

# Data:

Scan rescan of single subject on two different scanners with very different sequences 

Ptensor.nii.gz = prisma scanner, susceptibility corrected
Ttensor.nii.gz = trio scanner, not susceptibility corrected

## Get information about each volume 

In [29]:
! VolumeInfo 'Ptensor.nii.gz'

NIFTI Intent Code: NIFTI_INTENT_SYMMATRIX
NIFTI Orientation Code: RPI
Volume Info of Ptensor.nii.gz
size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]


In [30]:
! VolumeInfo 'Ttensor.nii.gz'

NIFTI Intent Code: NIFTI_INTENT_SYMMATRIX
NIFTI Orientation Code: RPI
Volume Info of Ttensor.nii.gz
size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]


# Preprocessing: 

(not sure why there's redundant functionality in TVAdjustVoxelspace and TVResample)

KMJ Note: 
- The tutorial recommends moving all volumes to origin [0,0,0] before processing. 
- If you want to do a diffeomorphic registration, make sure you resample such that the dimension 0 is a power of 2

## Adjust Voxel Space

- Output: New volume that has a new voxel size and/or origin (not: if output not specified, then the input volume is modified)
- Usage: TVAdjustVoxelspace -in tensor.nii.gz -out output.nii.gz -target target_tensor.nii.gz -vsize x y z -origin x y z

So you need to provide as input

- target: the filename of the template tensor you want matched in origin and voxel size

OR

- voxel size (vsize): the x y z sizes
    - DEFAULT not automatically passed
- origin: the x y z coordinates of the intended origin
    - DEFAULT {x,y,z} = {0,0,0}

NOTE: if you provide both a target tensor file and specify vsize and/or origin, then the tensor file space will be used

If you want to specify an output file, then you can override the default, which adds "_avs" to the end of the input file

### Tensor Volume

Usage of TVAdjustVoxelspace : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: N	by default use the input file name
 - -target        type: string(s)    required: N	output volume origin read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -origin        type: double(s)    required: N	

In [31]:
tv_avs_f = dtitk.utils.TVAdjustVoxSp()
tv_avs_f.inputs.in_file = fixed_tensor
tv_avs_f.inputs.origin = (0,0,0)
tv_avs_f.run()

180329-00:40:09,913 interface INFO:
	 stdout 2018-03-29T00:40:09.913058:Reading ./Ptensor.nii.gz ... Done in 0.054803s
180329-00:40:09,917 interface INFO:
	 stderr 2018-03-29T00:40:09.917476:WARNING: qform not defined
180329-00:40:09,919 interface INFO:
	 stderr 2018-03-29T00:40:09.917476:WARNING: assumed as RPI
180329-00:40:10,24 interface INFO:
	 stdout 2018-03-29T00:40:10.024166:input voxelspace : size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180329-00:40:10,25 interface INFO:
	 stdout 2018-03-29T00:40:10.025903:output voxelspace : size: 65x88x69, voxel size: 2x2x2, origin: [0, 0, 0]
180329-00:40:10,578 interface INFO:
	 stdout 2018-03-29T00:40:10.578279:Writing Ptensor_avs.nii.gz ... Done in 0.328118s


In [32]:
tv_avs_m = dtitk.utils.TVAdjustVoxSp()
tv_avs_m.inputs.in_file = moving_tensor
tv_avs_m.inputs.origin = (0,0,0)
tv_avs_m.run()

print(tv_avs_m.cmdline)

180329-00:40:10,819 interface INFO:
	 stdout 2018-03-29T00:40:10.819017:Reading ./Ttensor.nii.gz ... Done in 0.042166s
180329-00:40:10,828 interface INFO:
	 stderr 2018-03-29T00:40:10.828905:WARNING: qform not defined
180329-00:40:10,830 interface INFO:
	 stderr 2018-03-29T00:40:10.828905:WARNING: assumed as RPI
180329-00:40:10,930 interface INFO:
	 stdout 2018-03-29T00:40:10.930192:input voxelspace : size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180329-00:40:10,931 interface INFO:
	 stdout 2018-03-29T00:40:10.930192:output voxelspace : size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [0, 0, 0]
180329-00:40:11,302 interface INFO:
	 stdout 2018-03-29T00:40:11.302896:Writing Ttensor_avs.nii.gz ... Done in 0.225992s
TVAdjustVoxelspace -in ./Ttensor.nii.gz -origin 0 0 0 -out Ttensor_avs.nii.gz


### Scalar Volume

Usage of SVAdjustVoxelspace : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: N	by default use the input file name
 - -target        type: string(s)    required: N	output volume origin read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -origin        type: double(s)    required: N	

In [33]:
sv_avs_f = dtitk.utils.SVAdjustVoxSp()
sv_avs_f.inputs.in_file = fixed_vol
sv_avs_f.inputs.target_file = moving_vol
sv_avs_f.run()

print(sv_avs_f.cmdline)

180329-00:40:11,454 interface INFO:
	 stdout 2018-03-29T00:40:11.453992:Reading ./Ptensor_tr.nii.gz ... Done in 0.009084s
180329-00:40:11,462 interface INFO:
	 stdout 2018-03-29T00:40:11.462304:input voxelspace : size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180329-00:40:11,463 interface INFO:
	 stdout 2018-03-29T00:40:11.462304:reading output volume origin from ./Ttensor_tr.nii.gz
180329-00:40:11,465 interface INFO:
	 stdout 2018-03-29T00:40:11.465260:output voxelspace : size: 65x88x69, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180329-00:40:11,526 interface INFO:
	 stdout 2018-03-29T00:40:11.525995:Writing Ptensor_tr_avs.nii.gz ... Done in 0.055748s
SVAdjustVoxelspace -in ./Ptensor_tr.nii.gz -out Ptensor_tr_avs.nii.gz -target ./Ttensor_tr.nii.gz


In [34]:
sv_avs_m = dtitk.utils.SVAdjustVoxSp()
sv_avs_m.inputs.in_file = moving_vol
sv_avs_m.inputs.target_file = fixed_vol
sv_avs_m.run()

print(sv_avs_m.cmdline)

180329-00:40:11,689 interface INFO:
	 stdout 2018-03-29T00:40:11.688950:Reading ./Ttensor_tr.nii.gz ... Done in 0.006505s
180329-00:40:11,695 interface INFO:
	 stdout 2018-03-29T00:40:11.695826:input voxelspace : size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180329-00:40:11,698 interface INFO:
	 stdout 2018-03-29T00:40:11.695826:reading output volume origin from ./Ptensor_tr.nii.gz
180329-00:40:11,701 interface INFO:
	 stdout 2018-03-29T00:40:11.701485:output voxelspace : size: 62x80x60, voxel size: 2x2x2, origin: [-0, -0, -0]
180329-00:40:11,747 interface INFO:
	 stdout 2018-03-29T00:40:11.747761:Writing Ttensor_tr_avs.nii.gz ... Done in 0.043314s
SVAdjustVoxelspace -in ./Ttensor_tr.nii.gz -out Ttensor_tr_avs.nii.gz -target ./Ptensor_tr.nii.gz


## Resample 

- Output: New volume that has a new voxel size and/or origin (not: if output not specified, then the input volume is modified)
- Usage: TVAdjustVoxelspace -in tensor.nii.gz -out output.nii.gz -target target_tensor.nii.gz -vsize x y z -origin x y z

So you need to provide as input

- target: the filename of the template tensor you want matched

OR the specific parameters you want to change

- array size (size): the dimensions of the image
- voxel size (vsize): the x y z sizes
    - DEFAULT not automatically passed
- origin: the x y z coordinates of the intended origin
    - DEFAULT {x,y,z} = (0,0,0)

NOTE: if you provide both a target tensor file and specify vsize and/or origin, then the tensor file space will be used

In [35]:
tv_resample_f = dtitk.utils.TVResample()
tv_resample_f.inputs.in_file = fixed_tensor
tv_resample_f.inputs.origin = (0,0,0)
tv_resample_f.inputs.array_size= (128,128,64)
tv_resample_f.run()

print(tv_resample_f.cmdline)

180329-00:40:11,899 interface INFO:
	 stdout 2018-03-29T00:40:11.899581:interpolation option is LEI
180329-00:40:11,951 interface INFO:
	 stdout 2018-03-29T00:40:11.951851:Reading ./Ptensor.nii.gz ... Done in 0.050988s
180329-00:40:11,953 interface INFO:
	 stderr 2018-03-29T00:40:11.953256:WARNING: qform not defined
180329-00:40:11,955 interface INFO:
	 stderr 2018-03-29T00:40:11.953256:WARNING: assumed as RPI
180329-00:40:12,44 interface INFO:
	 stdout 2018-03-29T00:40:12.044505:input volume ./Ptensor.nii.gz
180329-00:40:12,46 interface INFO:
	 stdout 2018-03-29T00:40:12.044505:size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180329-00:40:12,116 interface INFO:
	 stdout 2018-03-29T00:40:12.115889:output volume specification: size: 128x128x64, voxel size: 2x2x2, origin: [0, 0, 0]
180329-00:40:12,362 interface INFO:
	 stdout 2018-03-29T00:40:12.362683:matching the center of the old and the new voxel spaces
180329-00:40:13,294 interface INFO:
	 stdout 2018-03-29T00:40:13.294155:Wr

In [36]:
tv_resample_m = dtitk.utils.TVResample()
tv_resample_m.inputs.origin = (0,0,0)
tv_resample_m.inputs.array_size= (128,128,64)
tv_resample_m.inputs.in_file=moving_tensor
tv_resample_m.run()

print(tv_resample_m.cmdline)

180329-00:40:13,540 interface INFO:
	 stdout 2018-03-29T00:40:13.540247:interpolation option is LEI
180329-00:40:13,603 interface INFO:
	 stdout 2018-03-29T00:40:13.603644:Reading ./Ttensor.nii.gz ... Done in 0.042414s
180329-00:40:13,605 interface INFO:
	 stderr 2018-03-29T00:40:13.605174:WARNING: qform not defined
180329-00:40:13,606 interface INFO:
	 stderr 2018-03-29T00:40:13.605174:WARNING: assumed as RPI
180329-00:40:13,715 interface INFO:
	 stdout 2018-03-29T00:40:13.715667:input volume ./Ttensor.nii.gz
180329-00:40:13,717 interface INFO:
	 stdout 2018-03-29T00:40:13.715667:size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180329-00:40:13,771 interface INFO:
	 stdout 2018-03-29T00:40:13.771315:output volume specification: size: 128x128x64, voxel size: 2.2x2.2x2.2, origin: [0, 0, 0]
180329-00:40:13,973 interface INFO:
	 stdout 2018-03-29T00:40:13.973886:matching the center of the old and the new voxel spaces
180329-00:40:14,762 interface INFO:
	 stdout 2018-03-29T00:4

In [37]:
sv_resample_f = dtitk.utils.SVResample()
sv_resample_f.inputs.in_file = fixed_vol
sv_resample_f.inputs.origin = (0,0,0)
sv_resample_f.inputs.array_size= (128,128,64)
sv_resample_f.run()

print(sv_resample_f.cmdline)

180329-00:40:14,956 interface INFO:
	 stdout 2018-03-29T00:40:14.956690:Reading ./Ptensor_tr.nii.gz ... Done in 0.009338s
180329-00:40:14,967 interface INFO:
	 stdout 2018-03-29T00:40:14.967260:input volume ./Ptensor_tr.nii.gz
180329-00:40:14,969 interface INFO:
	 stdout 2018-03-29T00:40:14.967260:size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180329-00:40:14,970 interface INFO:
	 stdout 2018-03-29T00:40:14.970565:output volume specification: size: 128x128x64, voxel size: 2x2x2, origin: [0, 0, 0]
180329-00:40:14,972 interface INFO:
	 stdout 2018-03-29T00:40:14.970565:matching the center of the old and the new voxel spaces
180329-00:40:15,184 interface INFO:
	 stdout 2018-03-29T00:40:15.184724:Writing Ptensor_tr_resampled.nii.gz ... Done in 0.07016s
SVResample -size 128 128 64 -in ./Ptensor_tr.nii.gz -origin 0 0 0 -out Ptensor_tr_resampled.nii.gz


In [38]:
sv_resample_m = dtitk.utils.SVResample()
sv_resample_m.inputs.origin = (0,0,0)
sv_resample_m.inputs.array_size= (128,128,64)
sv_resample_m.inputs.in_file = moving_vol
sv_resample_m.run()

print(sv_resample_m.cmdline)

180329-00:40:15,338 interface INFO:
	 stdout 2018-03-29T00:40:15.338378:Reading ./Ttensor_tr.nii.gz ... Done in 0.006267s
180329-00:40:15,344 interface INFO:
	 stdout 2018-03-29T00:40:15.344761:input volume ./Ttensor_tr.nii.gz
180329-00:40:15,346 interface INFO:
	 stdout 2018-03-29T00:40:15.344761:size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180329-00:40:15,349 interface INFO:
	 stdout 2018-03-29T00:40:15.348905:output volume specification: size: 128x128x64, voxel size: 2.2x2.2x2.2, origin: [0, 0, 0]
180329-00:40:15,351 interface INFO:
	 stdout 2018-03-29T00:40:15.348905:matching the center of the old and the new voxel spaces
180329-00:40:15,516 interface INFO:
	 stdout 2018-03-29T00:40:15.516810:Writing Ttensor_tr_resampled.nii.gz ... Done in 0.068091s
SVResample -size 128 128 64 -in ./Ttensor_tr.nii.gz -origin 0 0 0 -out Ttensor_tr_resampled.nii.gz


In [39]:
fixed_tensor_new = 'Ptensor_resampled.nii.gz'
moving_tensor_new = 'Ttensor_resampled.nii.gz'

## Calculate useful maps

### TVtool (NOTE: I haven't implemented all of them.. selected a few that are tested)

Usage of TVtool : 

 -in            type: string(s)    required: Y	
 -out           type: string(s)    required: N	
 -ad            type: flag         required: N	
 -rd            type: flag         required: N	
 -tr            type: flag         required: N	
 -fa            type: flag         required: N	
 -norm          type: flag         required: N	
 -dnorm         type: flag         required: N	
 -tsp           type: flag         required: N	
 -dtsp          type: flag         required: N	
 -dydisp        type: flag         required: N	
 -dycoh         type: flag         required: N	
 -lps           type: flag         required: N	
 -eigs          type: flag         required: N	
 -pdmp          type: flag         required: N	
 -faThreshold   type: double(s)    required: N	default: 0.2 
 -pd            type: flag         required: N	
 -rgb           type: flag         required: N	
 -luminance     type: double(s)    required: N	default: 1 	Adjusting RGB map luminance
 -faWeighting   type: integer(s)   required: N	default: 1 	Scaling by FA (1) or not (0)
 -spd           type: flag         required: N	
 -reorient      type: flag         required: N	
 -trans         type: string(s)    required: N	
 -euler         type: double(s)    required: N	
 -dev           type: flag         required: N	
 -dyadic        type: flag         required: N	
 -index         type: integer(s)   required: N	default: 0 
 -scale         type: double(s)    required: N	
 -add           type: string(s)    required: N	
 -subtract      type: string(s)    required: N	
 -mosaic        type: string(s)    required: N	
 -piece         type: integer(s)   required: N	default: 16 
 -aop           type: string(s)    required: N	
 -mask          type: string(s)    required: N	
 -alternate     type: double(s)    required: N	
 -nan           type: flag         required: N	
 -print         type: string(s)    required: N	
 -vsm           type: string(s)    required: N	
 -sm            type: string(s)    required: N	
 -SMOption      type: string(s)    required: N	default: EDS 	ED, DDS, EDS or GDS
 -orientation   type: string(s)    required: N	
 -replacement   type: string(s)    required: N	

In [40]:
tvt_f = dtitk.TVtool()
tvt_f.inputs.in_file = fixed_tensor_new
tvt_f.inputs.in_flag = 'tr'

tvt_f_res = tvt_f.run()
print(tvt_f.cmdline)

180329-00:40:15,778 interface INFO:
	 stdout 2018-03-29T00:40:15.777783:Reading Ptensor_resampled.nii.gz ... Done in 0.08528s
180329-00:40:16,69 interface INFO:
	 stdout 2018-03-29T00:40:16.069876:Computing the tensor trace (mean diffusion) map ... Done in 0.012845s
180329-00:40:16,183 interface INFO:
	 stdout 2018-03-29T00:40:16.183710:Writing Ptensor_resampled_tr.nii.gz ... Done in 0.063532s
TVtool -in Ptensor_resampled.nii.gz -tr -out Ptensor_resampled_tr.nii.gz


In [41]:
tvt_m = dtitk.TVtool()
tvt_m.inputs.in_file = moving_tensor_new
tvt_m.inputs.in_flag = 'tr'
tvt_m_res = tvt_m.run()
print(tvt_m.cmdline)

180329-00:40:16,454 interface INFO:
	 stdout 2018-03-29T00:40:16.454157:Reading Ttensor_resampled.nii.gz ... Done in 0.082552s
180329-00:40:16,768 interface INFO:
	 stdout 2018-03-29T00:40:16.768763:Computing the tensor trace (mean diffusion) map ... Done in 0.013301s
180329-00:40:16,869 interface INFO:
	 stdout 2018-03-29T00:40:16.869852:Writing Ttensor_resampled_tr.nii.gz ... Done in 0.052746s
TVtool -in Ttensor_resampled.nii.gz -tr -out Ttensor_resampled_tr.nii.gz


In [42]:
fixed_tr_new = 'Ptensor_resampled_tr.nii.gz'
moving_tr_new = 'Ttensor_resampled_tr.nii.gz'

### SVTool not implemented at this time

### Binarize Image (probably for mask to input to diffeo reg)

Usage:

BinaryThresholdImageFilter inputImage outputImage lower upper inside outside

- lower = lower bound threshold
- upper = upper bound threshold
- inside = value for voxels inside lower/upper bounds
- outside = value for voxels outside lower/upper bounds

KMJ NOTE: I'm not sure if you would want a decimal as the thresholds or inside/outside values. May want to change these to integers if that is unlikely

In [43]:
bt_f = dtitk.BinThresh()
bt_f.inputs.in_file = fixed_tr_new
bt_f.inputs.lower_bound = 0.01
bt_f.inputs.upper_bound=100
bt_f.inputs.inside_value=1
bt_f.inputs.outside_value=0
#bt.inputs.out_path = .replace('.nii.gz', '_trbmask.nii.gz')

bt_f.run()

print(bt_f.cmdline)

BinaryThresholdImageFilter Ptensor_resampled_tr.nii.gz Ptensor_resampled_tr_thrbin.nii.gz 0.01 100 1 0


In [44]:
bt_m = dtitk.BinThresh()
bt_m.inputs.in_file = moving_tr_new
bt_m.inputs.lower_bound = 0.01
bt_m.inputs.upper_bound=100
bt_m.inputs.inside_value=1
bt_m.inputs.outside_value=0
bt_m.run()
print(bt_m.cmdline)

BinaryThresholdImageFilter Ttensor_resampled_tr.nii.gz Ttensor_resampled_tr_thrbin.nii.gz 0.01 100 1 0


In [45]:
fixed_mask = 'Ptensor_resampled_tr_thrbin.nii.gz'
moving_mask = 'Ttensor_resampled_tr_thrbin.nii.gz'

# Alignment of DTI Volumes

## Rigid Registration

### dti_rigid_reg

- Output: Rigid alignment of a DTI volume (the subject) to a DTI template
- Usage: dti_rigid_reg template subject SMOption xsep ysep zsep ftol [useInTrans]

So you need to provide as input

- template: the filename of the template
- subject: the filename of the subject
- SMOption: the similarity metric you'd like to use for assessing the quality of alignment 
    - EDS: Euclidean Distance Squared between tensors (DEFAULT)
    - GDS: Geometric Distance Squared between tensors
    - DDS: Deviatoric Distance Squared between deviatoric tensors
    - NMI: Normalized Mutual Information between tensor components
- {x,y,z}sep: the distance between the sample points in the volume for the evaluation of image similarity (mm)
    - DEFAULT {x,y,z} = {2,2,2}
- ftol: the minimum amount of change in the cost function as a fraction of the previous value at the previous iteration for the registration optimization to stop
    - DEFAULT = 0.01
- useInTrans: the optional input to specify whether to initialize with some previously determined transformation.

The output from this command includes the registered image and the transformation that achieves the alignment. If we denote the filename of the subject as \${prefix}.nii.gz, then the filename of the registered image will be \${prefix_aff.nii.gz and that of the transformation will be \${prefix}.aff.

### Test interface

In [46]:
rigid = dtitk.Rigid()
rigid.inputs.fixed_file = fixed_tensor_new
rigid.inputs.moving_file = moving_tensor_new

rigid_res = rigid.run()
print(rigid.cmdline)

180329-00:40:17,588 interface INFO:
	 stdout 2018-03-29T00:40:17.585758:Rigid Registration of Ttensor_resampled.nii.gz to Ptensor_resampled.nii.gz
180329-00:40:17,599 interface INFO:
	 stdout 2018-03-29T00:40:17.585758:Similarity Measure : EDS
180329-00:40:17,602 interface INFO:
	 stdout 2018-03-29T00:40:17.585758:Reorient Option : FS
180329-00:40:17,670 interface INFO:
	 stdout 2018-03-29T00:40:17.670877:Reading Ptensor_resampled.nii.gz ... Done in 0.083168s
180329-00:40:17,950 interface INFO:
	 stdout 2018-03-29T00:40:17.950664:sampling step size = [2, 2, 2]
180329-00:40:18,405 interface INFO:
	 stdout 2018-03-29T00:40:18.405536:Gaussian smoothing: sigma = [0.736, 0.736, 0.736] ... Done in 0.423s
180329-00:40:18,506 interface INFO:
	 stdout 2018-03-29T00:40:18.505959:Reading Ttensor_resampled.nii.gz ... Done in 0.0753s
180329-00:40:19,195 interface INFO:
	 stdout 2018-03-29T00:40:19.195316:Gaussian smoothing: sigma = [0.645, 0.645, 0.645] ... Done in 0.442s
180329-00:40:19,535 interf

## Affine Registration

### dti_affine_reg

- Output: Affine alignment of a DTI volume (the subject) to a DTI template
- Usage: dti_affine_reg template subject SMOption xsep ysep zsep ftol [useInTrans]

So you need to provide as input

- template: the filename of the template
- subject: the filename of the subject
- SMOption: the similarity metric you'd like to use for assessing the quality of alignment 
    - EDS: Euclidean Distance Squared between tensors
    - GDS: Geometric Distance Squared between tensors
    - DDS: Deviatoric Distance Squared between deviatoric tensors
    - NMI: Normalized Mutual Information between tensor components
- {x,y,z}sep: the distance between the sample points in the volume for the evaluation of image similarity (mm)
- ftol: the minimum amount of change in the cost function as a fraction of the previous value at the previous iteration for the registration optimization to stop
- useInTrans: the optional input to specify whether to initialize with some previously determined transformation.

The output from this command includes the registered image and the transformation that achieves the alignment. If we denote the filename of the subject as \${prefix}.nii.gz, then the filename of the registered image will be \${prefix_aff.nii.gz and that of the transformation will be \${prefix}.aff.

### Test interface

In [47]:
affine = dtitk.Affine()
affine.inputs.fixed_file = fixed_tensor_new
affine.inputs.moving_file = moving_tensor_new
#Uncomment this line to use the rigid registration to initialize
affine.inputs.useInTrans = True

affine_res = affine.run()
print(affine.cmdline)

180329-00:40:26,690 interface INFO:
	 stdout 2018-03-29T00:40:26.690321:Affine Registration of Ttensor_resampled.nii.gz to Ptensor_resampled.nii.gz
180329-00:40:26,694 interface INFO:
	 stdout 2018-03-29T00:40:26.694194:Similarity Measure : EDS
180329-00:40:26,696 interface INFO:
	 stdout 2018-03-29T00:40:26.694194:Reorient Option : FS
180329-00:40:26,785 interface INFO:
	 stdout 2018-03-29T00:40:26.785808:Reading Ptensor_resampled.nii.gz ... Done in 0.083856s
180329-00:40:27,293 interface INFO:
	 stdout 2018-03-29T00:40:27.293224:sampling step size = [2, 2, 2]
180329-00:40:28,285 interface INFO:
	 stdout 2018-03-29T00:40:28.285353:Gaussian smoothing: sigma = [0.736, 0.736, 0.736] ... Done in 0.538s
180329-00:40:28,445 interface INFO:
	 stdout 2018-03-29T00:40:28.445071:Reading Ttensor_resampled.nii.gz ... Done in 0.0714s
180329-00:40:29,284 interface INFO:
	 stdout 2018-03-29T00:40:29.284102:Gaussian smoothing: sigma = [0.645, 0.645, 0.645] ... Done in 0.437s
180329-00:40:29,644 inter

## Apply Affine Transform to Tensor Volume

### affineSymTensor3DVolume

Usage of affineSymTensor3DVolume : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: N	specify an input transformation file; parameters input will be ignored
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -interp        type: string(s)    required: N	default: LEI 	EI or LEI
 - -reorient      type: string(s)    required: N	default: PPD 	NO, FS or PPD
 - -translation   type: double(s)    required: N	default: 0 0 0 	x y z in mm
 - -euler         type: double(s)    required: N	default: 0 0 0 	theta phi psi in degrees
 - -deformation   type: double(s)    required: N	default: 1 1 1 0 0 0 	xx yy zz xy yz xz


In [48]:
tv_ar = dtitk.registration.affSymTensor3DVol()
tv_ar.inputs.in_file = moving_tensor_new
tv_ar.inputs.transform = 'Ttensor_resampled.aff'

tv_ar.run()

print(tv_ar.cmdline)

180329-00:40:34,943 interface INFO:
	 stdout 2018-03-29T00:40:34.942989:inFile = Ttensor_resampled.nii.gz
180329-00:40:34,946 interface INFO:
	 stdout 2018-03-29T00:40:34.942989:outFile = Ttensor_resampled_affxfmd.nii.gz
180329-00:40:34,947 interface INFO:
	 stdout 2018-03-29T00:40:34.942989:interpolation option is LEI
180329-00:40:34,949 interface INFO:
	 stdout 2018-03-29T00:40:34.942989:reorientOption = PPD
180329-00:40:35,11 interface INFO:
	 stdout 2018-03-29T00:40:35.011434:Reading Ttensor_resampled.nii.gz ... Done in 0.06803s
180329-00:40:35,237 interface INFO:
	 stdout 2018-03-29T00:40:35.237279:trans (inverse) applied = [30.196, -9.065, 16.000]
180329-00:40:35,238 interface INFO:
	 stdout 2018-03-29T00:40:35.237279:[ 1.017, -0.137, -0.020;  0.141,  0.990,  0.132; -0.003, -0.143,  1.037]
180329-00:40:35,863 interface INFO:
	 stdout 2018-03-29T00:40:35.863730:backward resampling ...time consumed = 0.442
180329-00:40:36,600 interface INFO:
	 stdout 2018-03-29T00:40:36.599913:Writ

## Apply affine to Scalar Volume

### affineScalarVolume

Usage of affineScalarVolume : 
 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: N	specify an input transformation file; parameters input will be ignored
 - -interp        type: integer(s)   required: N	default: 0 	specify an interpolation scheme, 0 for trilinear (default) and 1 for nearest neighbor
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -translation   type: double(s)    required: N	default: 0 0 0 	x y z in mm
 - -euler         type: double(s)    required: N	default: 0 0 0 	theta phi psi in degrees
 - -deformation   type: double(s)    required: N	default: 1 1 1 0 0 0 	xx yy zz xy yz xz


In [49]:
sv_ar = dtitk.registration.affScalarVol()
sv_ar.inputs.in_file = moving_tr_new
sv_ar.inputs.transform = 'Ttensor_resampled.aff'

sv_ar.run()

print(sv_ar.cmdline)

180329-00:40:36,763 interface INFO:
	 stdout 2018-03-29T00:40:36.763758:inFile = Ttensor_resampled_tr.nii.gz
180329-00:40:36,767 interface INFO:
	 stdout 2018-03-29T00:40:36.763758:outFile = Ttensor_resampled_tr_affxfmd.nii.gz
180329-00:40:36,776 interface INFO:
	 stdout 2018-03-29T00:40:36.776387:Reading Ttensor_resampled_tr.nii.gz ... Done in 0.012325s
180329-00:40:36,804 interface INFO:
	 stdout 2018-03-29T00:40:36.804803:trans (inverse) applied = [30.196, -9.065, 16.000]
180329-00:40:36,806 interface INFO:
	 stdout 2018-03-29T00:40:36.804803:[ 1.017, -0.137, -0.020;  0.141,  0.990,  0.132; -0.003, -0.143,  1.037]
180329-00:40:36,903 interface INFO:
	 stdout 2018-03-29T00:40:36.903450:backward resampling ...time consumed = 0.097
180329-00:40:36,977 interface INFO:
	 stdout 2018-03-29T00:40:36.977309:Writing Ttensor_resampled_tr_affxfmd.nii.gz ... Done in 0.054s
affineScalarVolume -in Ttensor_resampled_tr.nii.gz -interp 0 -out Ttensor_resampled_tr_affxfmd.nii.gz -trans Ttensor_resamp

## Diffeomorphic Registration

### dti_diffeomorphic_reg

Output: Deformable alignment of a DTI volume (the subject) to a DTI template 

Usage: dti_diffeomorphic_reg template subject mask initial no_of_iter ftol

The required input are

- template: the filename of the template
- subject: the filename of the subject
- mask: the filename of a binary image that has 0 for background voxels and 1 for brain tissue voxels
- initial: a legacy parameter, just set to 1
- no_of_iter: number of iterations, recommended value to use is 6
- ftol: the minimum amount of change in the cost function as a fraction of the previous value at the previous iteration for the registration optimization to stop, recommended value is 0.002

The output from this command includes the registered image and the transformation that achieves the alignment. If we denote the filename of the subject as \${prefix}_aff.nii.gz, then the filename of the registered image will be \${prefix}_aff_diffeo.nii.gz and that of the transformation will be \${prefix}_aff_diffeo.df.nii.gz which is stored using the NIfTI vectorial format.

### Note

This is the point at which the dimension of the image matters (must be a power of 2). If you want to use linear registration only, then reslicing is not necessary. However, if you want to do nonlinear registration, then reslicing is required.

reslicing a mask didn't work for me... I had to create one in the new space using DTI-TK's tools (use TVtool to get trace, then put into BinaryThrehsoldImageFilter with params 0.01 100 1 0)

### Test interface

In [50]:
diffeo = dtitk.Diffeo()
diffeo.inputs.fixed_file = fixed_tensor_new
diffeo.inputs.moving_file = moving_tensor_new.replace('.nii.gz','_aff.nii.gz')
diffeo.inputs.mask_file = 'Ptensor_resampled_tr_thrbin.nii.gz'

diffeo_res = diffeo.run()

print(diffeo.cmdline)

180329-00:40:37,109 interface INFO:
	 stdout 2018-03-29T00:40:37.109019:registering Ttensor_resampled_aff.nii.gz to Ptensor_resampled.nii.gz ...
180329-00:40:37,113 interface INFO:
	 stdout 2018-03-29T00:40:37.113881:starting at Thu Mar 29 00:40:37 PDT 2018
180329-00:40:37,134 interface INFO:
	 stdout 2018-03-29T00:40:37.134418:
180329-00:40:37,136 interface INFO:
	 stdout 2018-03-29T00:40:37.134418:iteration 1 begins ...
180329-00:40:37,152 interface INFO:
	 stdout 2018-03-29T00:40:37.152465:Similarity Measure : DDS, Reorient Option : FS
180329-00:40:37,226 interface INFO:
	 stdout 2018-03-29T00:40:37.226354:Reading Ptensor_resampled.nii.gz ... Done in 0.072429s
180329-00:40:37,505 interface INFO:
	 stdout 2018-03-29T00:40:37.505796:Reading Ttensor_resampled_aff_diffeo.nii.gz ... Done in 0.061435s
180329-00:40:37,729 interface INFO:
	 stdout 2018-03-29T00:40:37.729677:Reading Ptensor_resampled_tr_thrbin.nii.gz ... Done in 0.004313s
180329-00:40:37,958 interface INFO:
	 stdout 2018-03-

180329-00:41:38,290 interface INFO:
	 stdout 2018-03-29T00:41:38.290378:iter 6: sum = 1250.89, img = 19076.4, prior = 534.516, reg = 1321.34
180329-00:41:41,398 interface INFO:
	 stdout 2018-03-29T00:41:41.398346:iter 7: sum = 1248.78, img = 19031.1, prior = 579.28, reg = 1224.71
180329-00:41:43,296 interface INFO:
	 stdout 2018-03-29T00:41:43.296598:iter = 65, iterGrad = 8
180329-00:41:43,297 interface INFO:
	 stdout 2018-03-29T00:41:43.296598:after : sum = 1247.68, img = 19000.3, prior = 577.277, reg = 1248.34
180329-00:41:43,299 interface INFO:
	 stdout 2018-03-29T00:41:43.299645:Little Endian System
180329-00:41:43,300 interface INFO:
	 stdout 2018-03-29T00:41:43.299645:Writing VECTORS ...
180329-00:41:44,261 interface INFO:
	 stdout 2018-03-29T00:41:44.260945:size is currently determined internally.
180329-00:41:44,262 interface INFO:
	 stdout 2018-03-29T00:41:44.260945:vsize = 1, 1, 1
180329-00:41:44,263 interface INFO:
	 stdout 2018-03-29T00:41:44.260945:origin is currently defa

180329-00:43:11,68 interface INFO:
	 stdout 2018-03-29T00:43:11.068432:iter 8: sum = 1154.11, img = 18605.1, prior = 154.447, reg = 663.982
180329-00:43:12,386 interface INFO:
	 stdout 2018-03-29T00:43:12.386838:iter 9: sum = 1152.71, img = 18696.3, prior = 142.812, reg = 502.139
180329-00:43:13,133 interface INFO:
	 stdout 2018-03-29T00:43:13.133897:iter = 67, iterGrad = 10
180329-00:43:13,135 interface INFO:
	 stdout 2018-03-29T00:43:13.133897:after : sum = 1152.38, img = 18725.8, prior = 132.846, reg = 468.672
180329-00:43:13,136 interface INFO:
	 stdout 2018-03-29T00:43:13.133897:Level 4
180329-00:43:13,138 interface INFO:
	 stdout 2018-03-29T00:43:13.138239:Piecewise Affine Setup: size = 32, 32, 16; vsize = 8, 8, 8
180329-00:43:13,140 interface INFO:
	 stdout 2018-03-29T00:43:13.140022:pDim = 196608; scalings : img = 0.06, prior = 0.09, reg = 0.036; ftol = 0.002
180329-00:43:13,272 interface INFO:
	 stdout 2018-03-29T00:43:13.272721:before : sum = 1152.38, img = 18724.4, prior = 1

180329-00:44:57,179 interface INFO:
	 stdout 2018-03-29T00:44:57.178935:Reading Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 0.650393s
180329-00:45:01,656 interface INFO:
	 stdout 2018-03-29T00:45:01.656241:backward resampling ...time consumed = 2.96539
180329-00:45:02,997 interface INFO:
	 stdout 2018-03-29T00:45:02.996991:Writing Ttensor_resampled_aff_diffeo_current.nii.gz ... Done in 0.73098s
180329-00:45:05,274 interface INFO:
	 stdout 2018-03-29T00:45:05.274715:IMAGE SIMILARITY: after previous iteration = 20922.8 after current iteration = 18651.9
180329-00:45:05,298 interface INFO:
	 stdout 2018-03-29T00:45:05.297993:iteration 2 done
180329-00:45:05,299 interface INFO:
	 stdout 2018-03-29T00:45:05.297993:
180329-00:45:05,300 interface INFO:
	 stdout 2018-03-29T00:45:05.300812:
180329-00:45:05,302 interface INFO:
	 stdout 2018-03-29T00:45:05.300812:iteration 3 begins ...
180329-00:45:05,320 interface INFO:
	 stdout 2018-03-29T00:45:05.320084:Similarity Measure

180329-00:46:08,580 interface INFO:
	 stdout 2018-03-29T00:46:08.580487:. 
180329-00:46:11,816 interface INFO:
	 stdout 2018-03-29T00:46:11.816444:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 2.49s
180329-00:46:11,949 interface INFO:
	 stdout 2018-03-29T00:46:11.949637:computing the deformation field: d2*d1: x -> d1(x) + d2(x + d1(x))
180329-00:46:12,398 interface INFO:
	 stdout 2018-03-29T00:46:12.398716:Reading Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 0.443223s
180329-00:46:14,28 interface INFO:
	 stdout 2018-03-29T00:46:14.028820:Reading Ttensor_resampled_aff_diffeo.df.nii.gz ... Done in 0.604427s
180329-00:46:26,522 interface INFO:
	 stdout 2018-03-29T00:46:26.522556:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 5.57855s
180329-00:46:26,742 interface INFO:
	 stdout 2018-03-29T00:46:26.741909:inFile = Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz
180329-00:46:27,399 interface INFO:
	 stdout

180329-00:47:45,732 interface INFO:
	 stdout 2018-03-29T00:47:45.732330:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 1.69881s
180329-00:47:45,820 interface INFO:
	 stdout 2018-03-29T00:47:45.820307:converting to the diffeomorphic deformation field: ...
180329-00:47:46,202 interface INFO:
	 stdout 2018-03-29T00:47:46.202715:Reading Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 0.380871s
180329-00:47:49,119 interface INFO:
	 stdout 2018-03-29T00:47:49.119059:Gaussian smoothing: sigma = [   1,    1,    1] ... Done in 1.97s
180329-00:47:51,72 interface INFO:
	 stdout 2018-03-29T00:47:51.072717:Gaussian smoothing: sigma = [   1,    1,    1] ... Done in 1.95s
180329-00:47:51,117 interface INFO:
	 stdout 2018-03-29T00:47:51.117316:maxNorm = 0.55	iterations = 1
180329-00:47:51,163 interface INFO:
	 stdout 2018-03-29T00:47:51.163589:Voxelwise scaling Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz by 0.5 ... Done in 0.046s
180329-00:47:5

180329-00:49:17,758 interface INFO:
	 stdout 2018-03-29T00:49:17.758564:Little Endian System
180329-00:49:17,759 interface INFO:
	 stdout 2018-03-29T00:49:17.758564:Writing VECTORS ...
180329-00:49:18,337 interface INFO:
	 stdout 2018-03-29T00:49:18.337859:size is currently determined internally.
180329-00:49:18,339 interface INFO:
	 stdout 2018-03-29T00:49:18.337859:vsize = 1, 1, 1
180329-00:49:18,340 interface INFO:
	 stdout 2018-03-29T00:49:18.337859:origin is currently default to [0, 0, 0].
180329-00:49:18,342 interface INFO:
	 stdout 2018-03-29T00:49:18.337859:outFile = Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz
180329-00:49:18,343 interface INFO:
	 stdout 2018-03-29T00:49:18.343837:Little Endian System
180329-00:49:18,346 interface INFO:
	 stdout 2018-03-29T00:49:18.343837:Reading VECTORS ...
180329-00:49:26,232 interface INFO:
	 stdout 2018-03-29T00:49:26.232329:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 1.76968s
180329-00:49:26,299 in

180329-00:51:12,527 interface INFO:
	 stdout 2018-03-29T00:51:12.526673:after : sum = 837.154, img = 13334.3, prior = 276.489, reg = 1163.53
180329-00:51:12,529 interface INFO:
	 stdout 2018-03-29T00:51:12.529337:Little Endian System
180329-00:51:12,530 interface INFO:
	 stdout 2018-03-29T00:51:12.529337:Writing VECTORS ...
180329-00:51:12,908 interface INFO:
	 stdout 2018-03-29T00:51:12.908198:size is currently determined internally.
180329-00:51:12,909 interface INFO:
	 stdout 2018-03-29T00:51:12.908198:vsize = 1, 1, 1
180329-00:51:12,911 interface INFO:
	 stdout 2018-03-29T00:51:12.908198:origin is currently default to [0, 0, 0].
180329-00:51:12,912 interface INFO:
	 stdout 2018-03-29T00:51:12.908198:outFile = Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz
180329-00:51:12,913 interface INFO:
	 stdout 2018-03-29T00:51:12.913837:Little Endian System
180329-00:51:12,915 interface INFO:
	 stdout 2018-03-29T00:51:12.913837:Reading VECTORS ...
180329-00:51:20,275 interface INFO:
	

TypeError: 'str' object is not callable

## Combine Transforms: Affine and Diffeomorphic 

### dfRightComposeAffine

Usage of dfRightComposeAffine :

 - -df            type: string(s)    required: Y	
 - -aff           type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	

### Test interface

In [51]:
combo = dtitk.ComposeXfm()
combo.inputs.in_df = 'Ttensor_resampled_aff_diffeo.df.nii.gz'
combo.inputs.in_aff = 'Ttensor_resampled.aff'

combo_res = combo.run()

print(combo.cmdline)

180329-00:53:55,996 interface INFO:
	 stdout 2018-03-29T00:53:55.996077:computing the deformation field: aff*df: x -> aff(x+df(x)) - x
180329-00:53:57,739 interface INFO:
	 stdout 2018-03-29T00:53:57.739685:Reading Ttensor_resampled_aff_diffeo.df.nii.gz ... Done in 1.69426s
180329-00:54:03,516 interface INFO:
	 stdout 2018-03-29T00:54:03.516005:Writing Ttensor_resampled_aff_diffeo.df_affdf.nii ... Done in 0.074633s
dfRightComposeAffine -aff Ttensor_resampled.aff -df Ttensor_resampled_aff_diffeo.df.nii.gz -out Ttensor_resampled_aff_diffeo.df_affdf.nii


TODO: Fix this interface. Setting the flag to keep the extension doesn't keep the whole thing (just df, not .nii or .nii.gz).

## Apply combined diffeo to tensor

### deformationSymTensor3DVolume

Usage of deformationSymTensor3DVolume : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: Y	
 - -df            type: string(s)    required: N	default: FD 
 - -interp        type: string(s)    required: N	default: LEI 	EI or LEI
 - -reorient      type: string(s)    required: N	default: PPD 	FS or PPD
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -flip          type: integer(s)   required: N	default: 0 0 0 
 - -type          type: integer(s)   required: N	default: 1 	resampling type, 1 for backward (default) and 0 for forward


In [52]:
tv_applydiff = dtitk.registration.diffeoSymTensor3DVol()
tv_applydiff.inputs.in_file = moving_tensor_new
tv_applydiff.inputs.transform = 'Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz'

tv_applydiff.run()

print(tv_applydiff.cmdline)

180329-00:54:03,699 interface INFO:
	 stdout 2018-03-29T00:54:03.698988:inFile = Ttensor_resampled.nii.gz
180329-00:54:03,702 interface INFO:
	 stdout 2018-03-29T00:54:03.698988:outFile = Ttensor_resampled_diffeoxfmd.nii.gz
180329-00:54:03,703 interface INFO:
	 stdout 2018-03-29T00:54:03.698988:transFile = Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz
180329-00:54:03,705 interface INFO:
	 stdout 2018-03-29T00:54:03.698988:df option is FD
180329-00:54:03,706 interface INFO:
	 stdout 2018-03-29T00:54:03.698988:interpolation option is LEI
180329-00:54:03,708 interface INFO:
	 stdout 2018-03-29T00:54:03.698988:reorientOption = PPD
180329-00:54:03,768 interface INFO:
	 stdout 2018-03-29T00:54:03.768725:Reading Ttensor_resampled.nii.gz ... Done in 0.068469s
180329-00:54:04,764 interface INFO:
	 stdout 2018-03-29T00:54:04.764751:Reading Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz ... Done in 0.72936s
180329-00:54:14,445 interface INFO:
	 stdout 2018-03-29T00:54:14.445123:backward resampling

## Apply combined diffeo to scalar volume

### deformationScalarVolume
Usage of deformationScalarVolume :

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: Y	
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -flip          type: integer(s)   required: N	default: 0 0 0 
 - -type          type: integer(s)   required: N	default: 1 	resampling type, 1 for backward (default) and 0 for forward
 - -interp        type: integer(s)   required: N	default: 0 	interpolation type, 0 for trilinear (default) and 1 for nearest neighbor

In [53]:
sv_applydiff = dtitk.registration.diffeoScalarVol()
sv_applydiff.inputs.in_file = moving_tr_new
sv_applydiff.inputs.transform = 'Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz'

sv_applydiff.run()

print(sv_applydiff.cmdline)

180329-00:54:18,220 interface INFO:
	 stdout 2018-03-29T00:54:18.220210:inFile = Ttensor_resampled_tr.nii.gz
180329-00:54:18,227 interface INFO:
	 stdout 2018-03-29T00:54:18.220210:outFile = Ttensor_resampled_tr_diffeoxfmd.nii.gz
180329-00:54:18,231 interface INFO:
	 stdout 2018-03-29T00:54:18.220210:transFile = Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz
180329-00:54:18,276 interface INFO:
	 stdout 2018-03-29T00:54:18.276450:Reading Ttensor_resampled_tr.nii.gz ... Done in 0.033821s
180329-00:54:20,696 interface INFO:
	 stdout 2018-03-29T00:54:20.696241:Reading Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz ... Done in 1.89271s
180329-00:54:22,973 interface INFO:
	 stdout 2018-03-29T00:54:22.973434:backward resampling ...time consumed = 0.25092
180329-00:54:23,52 interface INFO:
	 stdout 2018-03-29T00:54:23.052059:Writing Ttensor_resampled_tr_diffeoxfmd.nii.gz ... Done in 0.05228s
deformationScalarVolume -in Ttensor_resampled_tr.nii.gz -interp 0 -out Ttensor_resampled_tr_diffeoxfmd.ni

# Test pipeline